In [3]:
def recover_block(prev_cipher, cipher, is_first=False):
    """
    prev_cipher: block of ciphertext
    cipher: ciphertext to decrypt
    is_first: 
    
    returns decrypted message
    """
    
    # 1. Find Padding Lenght
    # Find padding length
    padding_length = 0
    for j in range(1,17):        
        # modify ith last block
        C_p_c = list(chunk(prev_cipher,1))
        # set jth last byte 
        C_p_c[-j] =  flip(C_p_c[-j])
        # reassemble sec last block
        C_p_flipped = b"".join(C_p_c)
        # assemble complete payload
        C_test = C_p_flipped + cipher
        r = oracle(C_test)

        print(f"Flipping [-{j}]:  {r}", end="\r")        
        if r:
            print(f"Predicted padding length: {j-1}")
            print(f"Padding: {bytes([j-1])*3}")
            padding_length = j-1
            break
    
    # 1. If padding length = 0, find 01
    
    
    ## 2+3 Determine last padding_length bytes of D(cipher)
    block_pad = chunk(prev_cipher[-padding_length:],1)
    recovered_m = [bytes([padding_length])]*padding_length    
    recovered_dec = []
    for m,c in zip(recovered_m, block_pad):
        recovered_dec.append(xor(m,c))
    
    ## 4 Loop
    for j in range(len(recovered_m) + 1,17):
        # Idx j is jth unresolved byte 
        ## Set new padding to j j ... j (j times)
        new_padding = [bytes([j])] * len(recovered_dec)
        # Construct new padding by xoring with recovered dec
        new_padding_xor = []
        for d,p in zip(recovered_dec,new_padding):
            new_padding_xor.append(xor(d,p))
        new_addoing_xor = joinhex(new_padding_xor)
        
        ## Create new message
        if j==1:
            # We need to brute force 01 first, ala padding was 0
            temp_p = list(chunk(prev_cipher[:], 1))
        else:
            # We have found previous paddings
            temp_p = list(chunk(prev_cipher[:-j + 1], 1))
        temp_p += new_padding_xor
        #print(hexify(temp_p))
        #print(len(temp_p))
        #print(hexify(temp_p[-j]))
        
        ## 5 Brute force jth position
        #print(hexify(temp_p[-j]))
        found_value = None
        for possible_value in range(256):
            # fresh temp_p
            new_prev_cipher = temp_p.copy()
            new_prev_cipher[-j] = bytes([possible_value])
            #print(hexify(new_prev_cipher))
            # check if valid
            new_ciphertext = joinhex(new_prev_cipher) + cipher
            r = oracle(new_ciphertext)
            if r:
                found_value = possible_value
                # We found value for dec
                dec_j = xor(bytes([j]), bytes([found_value]))
                recovered_dec.insert(0,dec_j)
                m_j = xor(dec_j,bytes([prev_cipher[-j]]))
                recovered_m.insert(0,m_j)
                print(f"Found dec {dec_j} and m {m_j}")
                break
        if found_value is None:
            print("Failed")
            break
    return recovered_m

In [7]:
import requests, re

url, auth = 'http://131.155.21.174:8083', ('', 'ilovecrypto')
sid, token = 20221886, 'ccfd8c7f0e9a20a30b999d76dbc4a9978d80c58e'

req = requests.Session()
req.cookies['token'] = token

r = req.get(url+'/', auth=auth)
ciphertext = bytes.fromhex(re.search('target_ciphertext = ([0-9a-f]+)', r.text).groups()[0])

def oracle(cipher):
    assert hasattr(cipher,'hex') and len(cipher) % 16 == 0 and len(cipher) >= 32
    r = req.post(url+'/oracle', auth=auth, data={'cipher': cipher.hex()})
    return bool(['False', 'True'].index(r.text.strip()))

def validate(plaintext):  # use this to submit your plaintext once you've decrypted it
    if type(plaintext) == bytes:
        try: plaintext = plaintext.decode()
        except UnicodeDecodeError: raise Exception('plaintext is printable ASCII; something is amiss.')
    assert type(plaintext) == str
    r = req.post(url+'/validate', auth=auth, data={'plaintext': plaintext})
    print('\x1b[32m{}\x1b[0m'.format(r.text.strip()))

################################################################

# let's try the oracle with a correct and invalid ciphertext.

res1 = oracle(ciphertext)
print('oracle (correct): {}'.format(res1))

# let's flip a bit
ciphertext_invalid = ciphertext[:-1] + bytes([ciphertext[-1] ^ 1])
res2 = oracle(ciphertext_invalid)
print('oracle (invalid): {}'.format(res2))

oracle (correct): True
oracle (invalid): False


In [8]:
block_size = 16

def xor(xs:bytes, ys: bytes) -> bytes:
    assert len(xs) == len(ys)
    return bytes(x^y for x,y in zip(xs,ys))

def pad(l):
    # Generate padding
    n = 16 - (l%16)
    return [int.to_bytes(n,1, "big")] * n

def chunk(l, n):
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]
        
def joinhex(byte_array):
    # Return a hex string from byte_array
    return b"".join(byte_array)

def hexify(byte_data):
    if isinstance(byte_data,list):
        return b"".join(byte_data).hex()
    if isinstance(byte_data,bytes):
        return byte_data.hex()
    # No match 
    raise ValueError(f"Invalid Type {type(byte_data)}")
    return b""
        

def flip(b):
    i = int.from_bytes(b,"big")
    return bytes([i^1])

In [9]:
C_c = list(chunk(ciphertext,16))

In [11]:
total_m = []
for i in reversed(range(1, len(C_c))):
    m = recover_block(C_c[i-1], C_c[i])
    total_m.append(m)

Predicted padding length: 3
Padding: b'\x03\x03\x03'
Found dec b'b' and m b'6'
Found dec b'/' and m b'd'
Found dec b'\xa8' and m b'4'
Found dec b'\x81' and m b'a'
Found dec b'w' and m b'b'
Found dec b'\xeb' and m b'd'
Found dec b'3' and m b'b'
Found dec b'P' and m b'a'
Found dec b'\xbc' and m b'a'
Found dec b'\xde' and m b'8'
Found dec b'\xa4' and m b'3'
Found dec b'!' and m b'2'
Found dec b'}' and m b'c'
Found dec b'\xa1' and m b'd'
Found dec b'\xec' and m b'9'
Found dec b'p' and m b'b'
Found dec b'\xa8' and m b'5'
Found dec b'\xda' and m b'c'
Found dec b'\x98' and m b'e'
Found dec b'\x01' and m b'4'
Found dec b']' and m b'a'
Found dec b'\xe6' and m b'f'
Found dec b'\xa7' and m b'7'
Found dec b'\xda' and m b'3'
Found dec b'\xe4' and m b'3'
Found dec b'\xe9' and m b'c'
Found dec b' ' and m b'0'
Found dec b'\xe2' and m b'd'
Found dec b'\x8c' and m b'd'
Found dec b'\x8f' and m b'7'
Found dec b'\xb8' and m b'4'
Found dec b'\x16' and m b'7'
Found dec b'A' and m b' '
Found dec b':' and m b'

In [12]:
m = b""
for m_block in reversed(total_m):
    for c in m_block:
        m +=c

In [13]:
m

b'Hello, 20221886. Randomness: 747dd0c337fa4ec5b9dc238aabdba4d6\x03\x03\x03'